# Hardware Crypto Attack


So far we have mostly been talking about software crypto. But how can we expand this to hardware crypto? Luckily it takes very few changes, so you don't have much to do!

In this lab we'll be looking at what is required to attack a hardware crypto device, and what sort of attacks work on these devices. In this case we're going to "cheat" and use an already recorded power trace, since we don't have hardware crypto on our target boards.

## Analysis

Now that we have our traces, we can begin our attack! We'll start off by importing everything we need for the attack:

In [ ]:
import chipwhisperer as cw
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, SBox_output, LastroundStateDiff

Next, we'll add our traces to a preprocessing module. We can feed `project.traceManager()` right into `attack.setTraceSource()`, but we could also add pre-processing inbetween (more about this later). We'll also re-open the traces, in this case it is required since the call to `closeAll()` would have flushed the buffers.

In [ ]:
#Force reload of project data (if you comment out 'closeAll()' this isn't needed)

#We also rebuild the project object in case you only want to run this half
project = cw.openProject('./tut_data/stm32f415_cpa_1k.cwp')

tm = project.traceManager()

This time we're going to do a few things. First we will get the traces, and plot a few of them as-is. You can adjust the traces plotted by adjusting the `range(10)`. For example `range(1)` plots the first trace.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_5 as palette
import itertools  

output_notebook()
p = figure(sizing_mode='scale_width', plot_height=300)

# create a color iterator
colors = itertools.cycle(palette)  

x_range = range(0, tm.numPoints())
for i, color in zip(range(10), colors): #Adjust range(n) to plot certain traces
    p.line(x_range, tm.getTrace(i), color=color)
show(p)

If this all works - let's just continue the attack! We're going to use the same leakge model as previously (Hamming weight), we'll seperate this out since will be changing that model around shortly.

In [ ]:
leak_model = AES128_8bit(SBox_output)

In [ ]:
attack = CPA()

attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(project.traceManager())
attack.setTraceStart(0)
attack.setTracesPerAttack(1000) #Only use a few traces for now...
attack.setIterations(1)
attack.setReportingInterval(50)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
attack.setPointRange((0, -1))

And then actually run it:

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

cb = cw.getJupyterCallback(attack)
attack_results = attack.processTracesNoGUI(stats_ready, show_progress_bar=True)

This will almost certainly fail. The leakage model is incorrect, so we need to find the correct (new) leakage model we should be using. This turns out to be pretty easy, since most typical hardware implementations use only one of a few possible models. We'll specifically just try the "Last Round State Over-Written" model first. You can do this by updating the model above to have the following:

In [ ]:
leak_model = AES128_8bit(LastroundStateDiff)

But this might not be enough! You may need to "window" around the area of interest. This is best done by plotting the results, and picking a nice area. For example a window of `attack.setPointRange((1380, 1390))` seems to work well on some traces.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

bnum = 4

key = tm.getKnownKey(0)[bnum]
data = attack.getStatistics().diffs[bnum]
xr = range(0, len(data[0]))

for v in range(0, 256):
    p.line(xr, data[v], line_color='green')

p.line(xr, data[key], line_color='red')
show(p)

You should see a graph of red and green in time (samples). In red is the correlation of the correct subkey for the first byte, while the rest are in green. You can use this graph to help fine-tune the windowing of the data.

## Conclusion

Attacking hardware crypto is similar to any other DPA style attack. In this example we have concentrated on the standard "Last Round State" to break a real hardware accelerator.